## Import Required Libraries

In [46]:
import re
import pandas as pd
import random
import spacy
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from spacy.training import Example
from spacy.tokens import Doc
from time import time

## Tagging Data
Format: ("Ingredient_Phrase", {"entities": [(start, end, "Ingredient"), (start, end,"Quantity"), (start, end, "Unit")]})
where start and end are starting and ending indexes respectively. 

In [47]:
tagged_entities = []
def tag_input(input_str):
    # Define the regex pattern
    pattern = r'(\S+)\s*(\d+(?:\.\d+)?)\s*(\S+)'

    # Extract the matches using the pattern
    matches = re.findall(pattern, input_str)

    # Create a list of tuples with the tagged entities
    for match in matches:
        ingredient = match[0]
        quantity = match[1]
        unit = match[2]
        entity = (input_str, {"entities": [(0, re.search('\d', input_str).start()-1, "Ingredient"), 
                                           (re.search('\d', input_str).start(), re.search('\d', input_str).end(), "Quantity"), 
                                           (re.search('\d', input_str).end()+1, len(input_str), "Unit")]})
        tagged_entities.append(entity)
#     return tagged_entities

In [48]:
df = pd.read_excel("urdu-recipes-ar.xlsx")
lst_ing_phrases = []
for index, row in df.iterrows():
    ing_name = row['ing_name']
    ing_quantity = row['Ing_quantity']
    ing_unit = row['Ing_unit']
    concatenated = str(ing_name) +" "+ str(ing_quantity) + " " +str(ing_unit)
#     print(concatenated)
    remove_nan = concatenated.replace("nan", "")
#     print(remove_nan)
    lst_ing_phrases.append(remove_nan)
for i in lst_ing_phrases:
    tag_input(i) 

### Let's look at "Tagged Entities"
tagged_entities = [
("ادرک لہسن کا پیسٹ 1 کھانے کا چمچہ", {"entities": [(0, 13, "Ingredient"), (14, 15, "Quantity"), (16, 23, "Unit")]}),
("ثابت لال مرچ 6 عدد", {"entities": [(0, 9, "Ingredient"), (10, 11, "Quantity"), (12, 15, "Unit")]}),
("ہری مرچ 3 عدد", {"entities": [(0, 7, "Ingredient"), (8, 9, "Quantity"), (10, 13, "Unit")]}),
("لہسن کے جوے آدھا کلو", {"entities": [(0, 11, "Ingredient"), (12, 13, "Quantity"), (14, 16, "Unit")]}),
("فِش فلے 10 عدد", {"entities": [(0, 7, "Ingredient"), (8, 10, "Quantity"), (11, 14, "Unit")]})]

## Shuffle and Split Data 

In [49]:
# Shuffle the data randomly
random.shuffle(tagged_entities)

# Split the data into train and test sets
train_data, test_data = train_test_split(tagged_entities, test_size=0.2, random_state=42)

## Initialize NER Model

In [50]:
# Initialize a blank Urdu model
nlp = spacy.blank("ur")

# Create a new NER component
ner = nlp.create_pipe("ner")

# Add the labels to the NER component
labels = ["Quantity", "Unit", "Ingredient"]
for label in labels:
    ner.add_label(label)    
    
# Add the NER component to the pipeline
nlp.add_pipe('ner', last=True)

## Train the Model

In [51]:
# Train the NER model
n_iter = 200
batch_size = 4
optimizer = nlp.begin_training()

start_time = time()
for i in range(n_iter):
    losses = {}
    random.shuffle(train_data)
    batches = minibatch(train_data, size=compounding(batch_size, 32, 0.051))
    for batch in batches:
        text, annotations = batch[0]
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
        epoch_loss = losses["ner"]
    print(f"Epoch {i+1} - Time taken: {time()-start_time:.2f} seconds")




Epoch 1 - Time taken: 0.05 seconds
Epoch 2 - Time taken: 0.11 seconds
Epoch 3 - Time taken: 0.16 seconds
Epoch 4 - Time taken: 0.21 seconds
Epoch 5 - Time taken: 0.26 seconds


C:\Users\HOME\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "تیل 0.25 کپ" with entities "[(0, 3, 'Ingredient'), (4, 5, 'Quantity'), (6, 11,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Epoch 6 - Time taken: 0.31 seconds
Epoch 7 - Time taken: 0.39 seconds
Epoch 8 - Time taken: 0.43 seconds
Epoch 9 - Time taken: 0.48 seconds


C:\Users\HOME\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "پسی ہوئی کالی مرچ 0.25 چائے کا چمچہ" with entities "[(0, 17, 'Ingredient'), (18, 19, 'Quantity'), (20,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\HOME\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "پانی  3 پیالی" with entities "[(0, 5, 'Ingredient'), (6, 7, 'Quantity'), (8, 13,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Epoch 10 - Time taken: 0.53 seconds
Epoch 11 - Time taken: 0.59 seconds
Epoch 12 - Time taken: 0.65 seconds
Epoch 13 - Time taken: 0.68 seconds
Epoch 14 - Time taken: 0.74 seconds


...

Epoch 139 - Time taken: 8.55 seconds
Epoch 140 - Time taken: 8.63 seconds
Epoch 141 - Time taken: 8.71 seconds
Epoch 142 - Time taken: 8.78 seconds
Epoch 143 - Time taken: 8.85 seconds
Epoch 144 - Time taken: 8.91 seconds
Epoch 145 - Time taken: 8.98 seconds
Epoch 146 - Time taken: 9.07 seconds
Epoch 147 - Time taken: 9.15 seconds
Epoch 148 - Time taken: 9.22 seconds
Epoch 149 - Time taken: 9.28 seconds
Epoch 150 - Time taken: 9.34 seconds
Epoch 151 - Time taken: 9.40 seconds
Epoch 152 - Time taken: 9.48 seconds
Epoch 153 - Time taken: 9.54 seconds
Epoch 154 - Time taken: 9.58 seconds
Epoch 155 - Time taken: 9.63 seconds
Epoch 156 - Time taken: 9.69 seconds


C:\Users\HOME\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "پیاز  1 عدد" with entities "[(0, 5, 'Ingredient'), (6, 7, 'Quantity'), (8, 11,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\HOME\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "پسا دھنیا  1 چائے کا چمچہ" with entities "[(0, 10, 'Ingredient'), (11, 12, 'Quantity'), (13,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Epoch 157 - Time taken: 9.75 seconds
Epoch 158 - Time taken: 9.81 seconds
Epoch 159 - Time taken: 9.87 seconds
Epoch 160 - Time taken: 9.93 seconds
Epoch 161 - Time taken: 9.98 seconds
Epoch 162 - Time taken: 10.05 seconds
Epoch 163 - Time taken: 10.10 seconds
Epoch 164 - Time taken: 10.15 seconds
Epoch 165 - Time taken: 10.18 seconds
Epoch 166 - Time taken: 10.24 seconds
Epoch 167 - Time taken: 10.30 seconds
Epoch 168 - Time taken: 10.35 seconds
Epoch 169 - Time taken: 10.39 seconds
Epoch 170 - Time taken: 10.44 seconds


C:\Users\HOME\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "سیلا چاول ½-2 پیالی" with entities "[(0, 11, 'Ingredient'), (12, 13, 'Quantity'), (14,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Epoch 171 - Time taken: 10.50 seconds
Epoch 172 - Time taken: 10.54 seconds
Epoch 173 - Time taken: 10.60 seconds
Epoch 174 - Time taken: 10.64 seconds
Epoch 175 - Time taken: 10.70 seconds
Epoch 176 - Time taken: 10.76 seconds
Epoch 177 - Time taken: 10.82 seconds
Epoch 178 - Time taken: 10.89 seconds
Epoch 179 - Time taken: 10.94 seconds
Epoch 180 - Time taken: 10.99 seconds
Epoch 181 - Time taken: 11.04 seconds
Epoch 182 - Time taken: 11.09 seconds
Epoch 183 - Time taken: 11.14 seconds
Epoch 184 - Time taken: 11.18 seconds
Epoch 185 - Time taken: 11.22 seconds
Epoch 186 - Time taken: 11.27 seconds
Epoch 187 - Time taken: 11.31 seconds
Epoch 188 - Time taken: 11.37 seconds
Epoch 189 - Time taken: 11.41 seconds
Epoch 190 - Time taken: 11.46 seconds
Epoch 191 - Time taken: 11.51 seconds
Epoch 192 - Time taken: 11.55 seconds
Epoch 193 - Time taken: 11.59 seconds
Epoch 194 - Time taken: 11.63 seconds
Epoch 195 - Time taken: 11.67 seconds
Epoch 196 - Time taken: 11.72 seconds
Epoch 197 - 

## Test the Model 

In [52]:
# Test the NER model on the test data
for text, annotations in test_data:
    doc = nlp(text)
    print("Text:", text)
    print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])
    print("")
# Save the model to disk
nlp.to_disk("urdu_ner_model")

Text: ادرک لہسن کا پیسٹ 1 چائے کا چمچہ
Entities: [('ادرک لہسن کا پیسٹ', 'Ingredient'), ('1', 'Quantity'), ('چائے کا چمچہ', 'Unit')]

Text: تیل 0.25 کپ
Entities: [('تیل 0.25 کپ', 'Ingredient')]

Text: مکئی کا آٹا 2 پیالی
Entities: [('مکئی کا آٹا', 'Ingredient'), ('2', 'Quantity'), ('پیالی', 'Unit')]

Text: پانی 1 کپ
Entities: [('پانی', 'Ingredient'), ('1', 'Quantity'), ('کپ', 'Unit')]

Text: پسا گرم مصالحہ 1 چائے کا چمچہ
Entities: [('پسا گرم مصالحہ', 'Ingredient'), ('1', 'Quantity'), ('چائے کا چمچہ', 'Unit')]

Text: پانی 1 ½ گلاس
Entities: [('پانی', 'Ingredient'), ('1', 'Quantity'), ('½', 'Unit'), ('گلاس', 'Ingredient')]

Text: ہری الائچی 4 عدد
Entities: [('ہری الائچی', 'Ingredient'), ('4', 'Quantity'), ('عدد', 'Unit')]

Text: پیاز  1 عدد
Entities: [('پیاز\xa0 ', 'Ingredient'), ('1', 'Quantity'), ('عدد', 'Unit')]

Text: سونٹھ 2 ٹکڑے
Entities: [('سونٹھ', 'Ingredient'), ('2', 'Quantity'), ('ٹکڑے', 'Unit')]

Text: رائی دانے 2 چائے کا چمچہ
Entities: [('رائی دانے', 'Ingredient'), ('2', 'Quan

Text: پسی کچی پیاز 2 عدد
Entities: [('پسی کچی پیاز', 'Ingredient'), ('2', 'Quantity'), ('عدد', 'Unit')]

Text: مسٹرڈ پیسٹ 3  کھانے کا چمچہ
Entities: [('مسٹرڈ پیسٹ', 'Ingredient'), ('3', 'Quantity')]

Text: پسی لال مرچ 1  کھانے کا چمچہ
Entities: [('پسی لال مرچ', 'Ingredient'), ('1', 'Quantity')]

Text: کارن فلور 1  کھانے کا چمچہ
Entities: [('کارن فلور', 'Ingredient'), ('1', 'Quantity')]

Text: کالی مرچ 1 چائے کا چمچہ
Entities: [('کالی مرچ', 'Ingredient'), ('1', 'Quantity'), ('چائے کا چمچہ', 'Unit')]

Text: ہری مرچ 3_4 عدد
Entities: [('ہری مرچ 3_4 عدد', 'Ingredient')]

Text: مرغی کا قیمہ 125 گرام
Entities: [('مرغی کا قیمہ', 'Ingredient'), ('125', 'Quantity')]

Text: لیموں 6 عدد
Entities: [('لیموں', 'Ingredient'), ('6', 'Quantity'), ('عدد', 'Unit')]

Text: تیل 1 پیالی
Entities: [('تیل', 'Ingredient'), ('1', 'Quantity'), ('پیالی', 'Unit')]

Text: کریم چیز  3  کھانے کا چمچہ
Entities: [('کریم چیز\xa0 ', 'Ingredient'), ('3', 'Quantity')]



### Let's look at a random test case.

In [53]:
doc = nlp('چنار کے پھول 1 کپ')
print("Text:", 'چنار کے پھول 1 کپ')
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])
print("")


Text: چنار کے پھول 1 کپ
Entities: [('چنار کے پھول', 'Ingredient'), ('1', 'Quantity'), ('کپ', 'Unit')]



## Evalution
### Metrics
* Precision
* Recall
* F1Score


In [54]:
# Function to evaluate the model on a test set
def evaluate_model(test_data):
    # Initialize counters
    total_true = 0
    total_predicted = 0
    total_correct = 0
    
    # Iterate over the test data
    for text, annotations in test_data:
        # Get the true entities
        true_entities = set([(ent[0], ent[1], ent[2]) for ent in annotations["entities"]])
        
        
        # Get the predicted entities
        doc = nlp(text)
        predicted_entities = set([(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents])
        
        # Update the counters
        total_true += len(true_entities)
        total_predicted += len(predicted_entities)
        total_correct += len(true_entities.intersection(predicted_entities))
    
    # Calculate precision, recall, and F1 score
    precision = total_correct / total_predicted if total_predicted > 0 else 0.0
    recall = total_correct / total_true if total_true > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
    
    # Print the evaluation metrics
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score: {f1_score:.2f}")

# Evaluate the model on the test data
evaluate_model(test_data)

Precision: 0.89
Recall: 0.82
F1 score: 0.86


### Conclusion: 
Above are the evaluation metrics of our NER model. The precision of the model is 0.89, which means that out of all the predicted entities, 89% were correct. The recall of the model is 0.82, which means that out of all the actual entities, the model was able to correctly identify 82% of them. The F1 score is 0.86, which is the harmonic mean of precision and recall and is a measure of the overall performance of the model which is indicating better performance.